In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

/tmp/ipykernel_10833/2011421623.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
## detection d'audio avec les extraits de 1min 
## avec la libraire scikit-maad 

# Correctly specifying your file path
path_audible_5s_train = "DataSet/CSVs_morceaux_audio_5s/Audible/train_audible_recording_chunks.csv"
path_audible_5s_test = "DataSet/CSVs_morceaux_audio_5s/Audible/test_audible_recording_chunks.csv"
path_audible_5s_val = "DataSet/CSVs_morceaux_audio_5s/Audible/val_audible_recording_chunks.csv"

# Using pandas to read the CSV file into a DataFrame
df_train = pd.read_csv(path_audible_5s_train)
df_test = pd.read_csv(path_audible_5s_test)
df_val = pd.read_csv(path_audible_5s_val)

# Paths for the audio files 
path_global_5s_train = "DataSet/Selection_morceaux_audio_5s/Audible/train_audible_recording_chunks.csv"
path_global_5s_test = "DataSet/CSVs_morceaux_audio_5s/Audible/test_audible_recording_chunks.csv"
path_global_5s_val = "DataSet/CSVs_morceaux_audio_5s/Audible/val_audible_recording_chunks.csv"

In [ ]:
# Path Yolo results
path_audible_results_YOLO = "DataSet/soundscape_to_5s/Selection soundscapes 1 min/"
path_audible_csv_YOLO = "DataSet/soundscape_to_5s/CSV soundscapes 1 min split 5s boxes/1min_to_5s_boxes.csv"
df_yolo = pd.read_csv(path_audible_csv_YOLO)

soundscape_truth_df = "DataSet/CSV soundscapes 1 min/selected_soundscapes.csv"
# generer spectrogramme 
soundscape_image_path = "DataSet/Selection soundscapes 1 min/"

df_truth = pd.read_csv(soundscape_truth_df)

In [ ]:
# Comparer les bounding boxes 

